1. compile oofem with -DUSE_TRACE_FIELDS
2. adjust paths to the build dir below
3. run all the remaining cells to get converted inputs in the `xml` directory
4. symlink the `./xml` directory to the `$OOFEM_DIR/tests/xml`, ctest will pick XML tests up automatically

What does not work: xfem, contacts (need special records), a few others for unclear reasons.

In [1]:
import os
OOFEM_DIR='/home/eudoxos/oofem'
FIELDS_CSV=os.getcwd()+'/FIELDS.csv'
!rm -f {FIELDS_CSV}
!OOFEM_TRACE_FIELDS_CSV={FIELDS_CSV} ctest --test-dir {OOFEM_DIR}/build-eigen/ --parallel 16  --progress

Internal ctest changing into directory: /home/eudoxos/oofem/build-eigen
Test project /home/eudoxos/oofem/build-eigen
  1/401 Test  #75: test_sm_control_switch_2.in.........................***Exception:   0.30 sec
401/401 Test #391: benchmark_fm_bdam7.oofem.ind.inin[Km.in
99% tests passed, 1 tests failed out of 401

Total Test time (real) =  16.32 sec

The following tests FAILED:
	 75 - test_sm_control_switch_2.in (Subprocess aborted)
Errors while running CTest
Output from these tests are in: /home/eudoxos/oofem/build-eigen/Testing/Temporary/LastTest.log
Use "--rerun-failed --output-on-failure" to re-run the failed cases verbosely.


In [2]:
from rich.pretty import pprint
import json
ATTRS={}
ENUMS={}
import csv
for ir,(tag,att,T) in enumerate(csv.reader(open(FIELDS_CSV,'r'),delimiter=';')):
    tag=tag.lower()
    if tag=='~enum~':
        if att not in ENUMS: ENUMS[att]=dict([(int(k),v) for k,v in json.loads(T).items()])
        continue
    if tag not in ATTRS: ATTRS[tag]={}
    D=ATTRS[tag]
    if att in D:
        if T=='flag': continue
        if T=='std::string' and D[att].startswith('enum:'): continue
        if D[att]=='flag' or (D[att]=='std::string' and T.startswith('enum:')):
            D[att]=T
            continue
        if D[att]!=T: print(f'WARN: inconsistent type {tag}.{att}: {D[att]} ≠ {T}')
    else: D[att]=T

for CH in ['NODE','ELEMENT','BEAM_ELEMENT','REACTION','LOADLEVEL','EIGVAL','TIME']:
    ATTRS['ch_'+CH.lower()]={'tstep':'int','tstepver':'int','number':'int','dofman':'int','dof':'int','irule':'int','gp':'int','component':'int','record':'int','keyword':'std::string','unknown':'std::string','value':'double','eignum':'int','tolerance':'double'}
ATTRS['ch_begin_checks']={'tolerance':'double'}

WARN: inconsistent type frcfcmnl.m: int ≠ double


In [3]:
import json
import pydantic
import re
from rich.pretty import pprint
from typing import List
from lxml import etree
import csv

class InLine(pydantic.BaseModel):
    '''One input line, with location information'''
    f: str
    lineno: int
    raw: str
    comments: List[str]
    toks: List[str]=[]
    def loc(self): return f'{self.f}:{self.lineno}'
class oofemDialect(csv.Dialect):
    'Used to tokenize one line (processes "...")'
    delimiter=' '
    quotechar='"'
    quoting=csv.QUOTE_MINIMAL
    skipinitialspace=True
    strict=True
    lineterminator=''
def readOofem(filename):
    def fixComment(c):
        if c.endswith('-'): c=c+' '
        return c.replace('--','—')
    file=open(filename,'r')
    comment=[]
    checkMode=False
    lines=[]
    checks=[]
    checkParams=''
    for il,l in enumerate(file.readlines()):
        l=l.removesuffix('\n').replace('\t',' ')
        if (m:=re.fullmatch('#%BEGIN_CHECK%(.*)',l)) and not checkMode :
            if checks:
                print(f'{filename}:{il+1}: WARN: more than one BEGIN_CHECK blocks, ignoring them.')
                comment.append(l[1:])
                continue
            checkMode=True
            rest=m.group(1).strip()
            if len(checks)==0: checks.append(InLine(f=filename,lineno=il+1,raw=rest,comments=comment[:]))
            else:
                # if there are multiple BEGIN_CHECK, concatenate params and comments to the first one
                if rest: checks[0].raw+=' '+rest
                checks[0].comments+=comment
            comment=[]
            continue      
        if l.startswith('#%END_CHECK%') and checkMode:
            checkMode=False
            if comment: checks[-1].comments+=comment
            comment=[]
            continue
        if checkMode:
            if l=='': continue
            if l[0]!='#':
                print(f'{filename}:{il+1}: WARN: check line not starting with #: "{l}"')
                # raise ValueError(f'{filename}:{il+1}: check line not starting with #: "{l}"')
            elif '//' in l:
                a,b=l[1:].split('//',maxsplit=1)
                l=a
                comment.append(b)
            else: l=l[1:]
        if m:=re.fullmatch(r'\s*',l): continue
        if m:=re.fullmatch(r'\s*#+\s*',l): continue
        if m:=re.fullmatch(r'\s*#+(.*)',l):
            comment.append(fixComment(m.group(1).strip()))
            continue
        if m:=re.fullmatch(r'^(.*)#+(.*)',l):
            l=m.group(1)+"\n" # stripped just below
            comment.append(fixComment(m.group(2).strip()))
        l=l.strip()
        if checkMode: checks.append(InLine(f=filename,lineno=il+1,raw='CH_'+l,comments=comment[:]))
        else: lines.append(InLine(f=filename,lineno=il+1,raw=l,comments=comment[:]))
        comment=[]
    # add leftover comments to whatever was last
    if comment:
        #if not checks or lines[-1].lineno>checks[-1].lineno: lines[-1].comments+=comment
        #else: checks[-1].comments+=comment
        (lines[-1] if (not checks or lines[-1].lineno>checks[-1].lineno) else checks[-1]).comments+=comment
    for LL in lines,checks:
        toks=csv.reader([l.raw for l in LL],oofemDialect)
        for l,t in zip(LL,toks): l.toks=t
    return lines,checks

def getNocase(d,k):
    if k in d: return d[k]
    fromlower=dict([(k.lower(),v) for k,v in d.items()])
    return fromlower.get(k.lower(),None)
def fixCase(d,k):
    if k in d: return k
    fromlower=dict([(k.lower(),k) for k in d.keys()])
    return fromlower.get(k,None)
        

def readNextAttr(T,toks,loc,line):
    'consume toks, return (attribute name,value)'
    attr=toks.pop(0).lower()
    def pop0N(n=None,cast=None):
        if n is None: n=int(toks.pop(0))
        if cast is None: return [toks.pop(0) for i in range(0,n)]
        return [cast(toks.pop(0)) for i in range(0,n)]

    if T in ('errorcheck','vtkxml') and attr in ('1','2'): return None,None
    
    if T not in ATTRS: raise ValueError(f'{loc}: {T}: no such record type')
    if attr not in ATTRS[T]: raise ValueError(f'{loc}: {T}.{attr}: no such attribute')
    aT=ATTRS[T][attr]    
        
    if aT=='int': return attr,int(toks.pop(0))
    elif aT=='flag':
        # ATTRS_FLAGS_INT_MAYBE
        if len(toks)>0 and toks[0].isdigit():
            print(f'{loc}: flag {T}.{attr} followed by an extra numerical field {toks[0]}')
            toks.pop(0)
        return attr,'' # None
    elif aT=='bool': return attr,{True:'1',False:'0'}[bool(toks.pop(0))]
    elif aT=='double': return attr,float(toks.pop(0))
    elif aT=='IntArray': return attr,' '.join([str(i) for i in pop0N(cast=int)])
    elif aT=='FloatArray': return attr,' '.join([str(i) for i in pop0N(cast=float)])
    elif aT=='std::string': return attr,toks.pop(0)
    elif aT=='ScalarFunction':
        if not toks[0].startswith('$'): return attr,toks.pop(0)
        tt=[]
        while True:
            tt.append(t:=toks.pop(0)) # .removeprefix('$').removesuffix('$'))
            if t.endswith('$'): break
        return attr,' '.join(tt)
    elif aT=='Dictionary':
        sz=int(toks.pop(0))
        d=dict([(toks.pop(0),toks.pop(0)) for i in range(sz)])
        return attr,'; '.join([f'{k} {v}' for k,v in d.items()])
    elif aT=='FloatMatrix':
        rows,cols=int(toks.pop(0)),int(toks.pop(0))
        assert toks[0].startswith('{')
        vv=[]
        while True:
            vv.append((t:=toks.pop(0)).removeprefix('{').removesuffix(';').removesuffix('}'))
            if t.endswith('}'): break
        vv=[float(v) for v in vv if v!='']
        assert len(vv)==rows*cols
        # separate rows by semicolon
        return attr,'; '.join([' '.join([str(v) for v in vv[row*cols:(row+1)*cols]]) for row in range(rows)])
    elif aT=='std::list<Range>':
        assert toks[0][0]=='{'
        tt=[]
        while True:
            tt.append(toks.pop(0))
            if tt[-1].endswith('}'): break
        s=' '.join(tt).removeprefix('{').removesuffix('}')
        s=re.sub(r'\(\s*([0-9]+)\s+([0-9]+)\s*\)',r'\1-\2',s).strip()
        s=re.sub(r'\s+',',',s)
        return attr,s
    elif aT.startswith('enum:'):
        E=ENUMS[aT.removeprefix('enum:')]
        return attr,sorted(E[int(toks.pop(0))],key=lambda x: len(x))[0] # return the shortest possible literal for the value
    else: raise ValueError(f'{loc}: {T}.{attr} has unhandled type {aT=}.\n{toks}')
    
def readInstance(line,optionalT=None,T=None,hasId=False):
    # print(line)
    ret={'_c':line.comments}
    toks=line.toks[:]
    if optionalT:
        if toks[0].lower()==optionalT.lower(): toks.pop(0)
    if T is None: T=toks.pop(0)
    ret['_T']=T
    if hasId: ret['_id']=int(toks.pop(0))
    while toks:
        # print(toks)
        try: k,v=readNextAttr(T.lower(),toks,loc=line.loc(),line=line)
        except BaseException as e: raise ValueError(f'{line.loc()}: error reading attribute: {str(e)}')
        # ignored attribute (used for nonsense such as 'errorcheck 1'
        if k is None: continue
        # print(f'{k=} {v=}')
        ret[k]=v
    return ret

# pprint(readOofem('../tests/sm/control_switch_1.in'))

In [4]:
from enum import Enum
REC = Enum('REC','''output jobdesc analysis metasteps initmodules exportmodules mpmvariables mpmterms mpmintegrals fields
    domains outputmanager domaincompres
    nodes elements crosssections materials nonlocalbarriers boundaryconditions initialconditions loadtimefunctions sets 
    checks'''.split())

# TODO: xfem
#    - xfemmanager enrichmentfunction geometry enrichmentitem enrichmentfront propagationlaw cracknucleation fracturemanager failcriterion

def parseOofem(filename):
    lines,checks=readOofem(filename)
    # pprint(checks)
    pb={}
    # 1. output file record
    pb[REC.output]=lines.pop(0).raw
    
    # 2. job description
    pb[REC.jobdesc]=lines.pop(0).raw
    
    # 3. analysis
    analysisType=lines[0].toks[0]
    pb[REC.analysis]=ana=readInstance(lines.pop(0),T=None)
    # pprint(pb[3])
    pb[REC.metasteps]=[readInstance(lines.pop(0),T='~MetaStep~') for i in range(ana.pop('nmsteps',0))]
    pb[REC.initmodules]=[readInstance(lines.pop(0)) for i in range(ana.pop('ninitmodules',0))]
    pb[REC.exportmodules]=[readInstance(lines.pop(0)) for i in range(ana.pop('nmodules',0))]

    # MPM-specific
    pb[REC.mpmvariables]=[readInstance(lines.pop(0)) for i in range(ana.pop('nvariables',0))]
    pb[REC.mpmterms]=[readInstance(lines.pop(0),hasId=True) for i in range(ana.pop('nterms',0))]
    pb[REC.mpmintegrals]=[readInstance(lines.pop(0),hasId=True) for i in range(ana.pop('nintegrals',0))]

    # fields
    pb[REC.fields]=[readInstance(lines.pop(0),hasId=False) for i in range(ana.pop('nfields',0))]


    setsAfterElements=False

    # 4. domains
    # handle case-by-case basis
    ndomains=1 
    assert lines[0].toks[0].lower()=='domain'
    pb[REC.domains]=[lines.pop(0).toks for i in range(ndomains)]
    
    # 5. OutputManager
    pb[REC.outputmanager]=readInstance(lines.pop(0),T='~OutputManager~',optionalT='OutputManager')
    
    # 6. component size record
    pb[REC.domaincompres]=dcr=readInstance(lines.pop(0),T='~DomainCompRec~')
    
    # 7. node records
    pb[REC.nodes]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr['ndofman'])]
    # 8. element records
    pb[REC.elements]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr['nelem'])]
    # older syntax order
    if lines[0].toks[0].lower()=='set':
        setsAfterElements=True
        pb[REC.sets]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr.get('nset',0))]
    # 10. cross sections
    pb[REC.crosssections]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr['ncrosssect'])]
    # 11. material records
    pb[REC.materials]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr['nmat'])]
    # 12. nonlocal barriers
    pb[REC.nonlocalbarriers]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr.get('nbarrier',0))]

    # 13. load/boundary conditions
    pb[REC.boundaryconditions]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr['nbc'])]
    # 14. initial conditions
    pb[REC.initialconditions]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr['nic'])]
    # 15. time function records
    pb[REC.loadtimefunctions]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr['nltf'])]
    # 9. set records
    if not setsAfterElements:
        pb[REC.sets]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr.get('nset',0))]

    # pb[REC.xfemmanagersloadtimefunctions]=[readInstance(lines.pop(0),hasId=True) for i in range(dcr['nltf'])]

        
    if checks: pb[REC.checks]=[readInstance(checks[0],T='ch_begin_checks')]+[readInstance(l) for l in checks[1:]]

    return pb

if 0:
    # parseOofem('../tests/sm/deepbeamFE2_01.in')
    # pprint(p)
    import glob
    # for f in sorted(glob.glob('../tests/sm/*.in')):
    for f in ['../tests/sm/control_switch_1.in']:
        # print(f)
        pb=parseOofem(f)
        # pprint(pb)

In [5]:
def pb2xml(pb):
    from lxml import etree
    root=etree.Element("oofem",version="1")
    def app(parent,tag,_lastId=0,text=None,_c=[],_T=None,comments=[],**kw):
        if tag is None: tag=_T
        tag={'set':'Set','node':'Node'}.get(tag,tag)
        kw.pop('_T',None)
        for c in comments+_c: parent.append(etree.Comment(c))
        if '_id' in kw:
            if (id:=kw.pop('_id'))%5==0 or id!=_lastId+1: kw={'id':id}|kw
            _lastId=id
        def fixKw(k):
            return {'tstep':'tStep'}.get(k.lower(),k.lower()).replace('/','_').replace('(','_').replace(')','_')
        kw2=dict([(fixKw(k),str(v)) for k,v in kw.items()])
        if 'f(t)' in kw2.keys(): kw2['f_t_']=kw2.pop('f(t)')
        parent.append(e:=etree.Element(tag,**kw2))
        if text is not None: e.text=text
        return e,_lastId
    app(root,'Output',text=pb[REC.output])
    app(root,'Description',text=pb[REC.jobdesc])
    # analysis
    ana=app(root,tag='Analysis',type=pb[REC.analysis].pop('_T'),**pb[REC.analysis])[0]
    # metasteps
    if REC.metasteps in pb and len(pb[REC.metasteps])>0:
        ana.append(ms:=etree.Element('Metasteps'))
        for n in pb[REC.metasteps]: app(ms,tag='Metastep',**n)
    # MPM
    if REC.mpmvariables in pb and len(pb[REC.mpmvariables])>0:
        for group,key,tag in [('MPMVariables',REC.mpmvariables,'Variable'),('MPMTerms',REC.mpmterms,None),('MPMIntegrals',REC.mpmintegrals,'Integral')]:
            ana.append(gr:=etree.Element(group))
            lastId=0
            for n in pb[key]: _,lastId=app(gr,_lastId=lastId,tag=None,**n)
    if REC.fields in pb and len(pb[REC.fields])>0:
        ana.append(gr:=etree.Element('Fields'))
        for n in pb[REC.fields]: app(gr,tag=None,**n)
        
    # root.append(domains:=etree.Element('Domains'))
    for domRec in pb[REC.domains]:
        root.append(dom:=etree.Element('Domain',domain=domRec[1].lower()))
        app(dom,'OutputManager',**pb[REC.outputmanager])
        for group,key in [('Nodes',REC.nodes),('Elements',REC.elements),('CrossSections',REC.crosssections),('Materials',REC.materials),('NonlocalBarriers',REC.nonlocalbarriers),('BoundaryConditions',REC.boundaryconditions),('InitialConditions',REC.initialconditions),('LoadTimeFunctions',REC.loadtimefunctions),('Sets',REC.sets)]:
            if group in ('NonlocalBarriers',) and len(pb[key])==0: continue
            dom.append(nodes:=etree.Element(group))
            lastId=0
            for n in pb[key]: _,lastId=app(nodes,_lastId=lastId,tag=None,**n)
            
    for group,key in [('InitModules',REC.initmodules),('ExportModules',REC.exportmodules)]:
        if len(pb.get(key,[]))==0: continue
        root.append(mm:=etree.Element(group))
        for n in pb[key]:
            if n['_T'].lower()=='errorcheck' and (REC.checks in pb):
                n|=pb[REC.checks][0]
                n['_T']='errorcheck' # lowercase
            mod=app(mm,tag=None,**n)[0]
            if n['_T'].lower()=='errorcheck':
                for c in pb[REC.checks][1:]:
                    c['_T']=c['_T'].removeprefix('ch_').removeprefix('CH_').upper()
                    app(mod,tag=None,**c)
    return root


In [6]:
if 1:
    import glob, os, shutil
    testDir=f'{OOFEM_DIR}/tests/'
    for root0,dirs,files in os.walk(testDir):
        #for testDir in ['sm','tm','fm']:
        root=root0.removeprefix(testDir)
        if root.startswith('xml'): continue
        if root.startswith('Testing'): continue
        if 'python' in root: continue
        dstDir=f'./xml/{root}'
        # if root!='mpm': continue
        if root.split('/')[0] in ('smmfront','partests','tmcemhyd','fmpfem','tmsm','am'): continue
        print(root)
        os.makedirs(dstDir,exist_ok=True)
        for f0 in sorted(files):
            # print(f'{root0}/{f0} → {dstDir}/{f0}')
            f=f'{root0}/{f0}'
            ext=os.path.splitext(f)[1]
            # print(f)
            # for f in sorted(glob.glob(f'../tests/{testDir}/*')):
            # for f in ['../tests/sm/lattice3dboundarytruss.in']:
            #if os.path.isdir(f): continue
            if ext=='.in':
                out=f'{dstDir}/{os.path.basename(f).removesuffix(".in")}.xml'
                print(f'{f} → {out}')
                try:
                    xml=pb2xml(pb:=parseOofem(f))
                    open(out,'wb').write(etree.tostring(xml,pretty_print=True))
                except Exception as e:
                    print(f'{f}: error:\n    {str(e)}')
                    # raise
            elif ext in ('.rve','.dat','.t3d'):
                dst=f'{dstDir}/{os.path.basename(f)}'
                print(f'   {f} → {dst}')
                shutil.copy(f,dst)
            else:
                continue


fm
/home/eudoxos/oofem/tests/fm/cbs1.in → ./xml/fm/cbs1.xml
/home/eudoxos/oofem/tests/fm/cbs2.in → ./xml/fm/cbs2.xml
/home/eudoxos/oofem/tests/fm/cbs3.in → ./xml/fm/cbs3.xml
/home/eudoxos/oofem/tests/fm/scctest01.in → ./xml/fm/scctest01.xml
/home/eudoxos/oofem/tests/fm/simpleNonlinearStokes.in → ./xml/fm/simpleNonlinearStokes.xml
/home/eudoxos/oofem/tests/fm/weakPeriodicTriangularObstacle.in → ./xml/fm/weakPeriodicTriangularObstacle.xml
sm
/home/eudoxos/oofem/tests/sm/Buckling01.in → ./xml/sm/Buckling01.xml
/home/eudoxos/oofem/tests/sm/Buckling02.in → ./xml/sm/Buckling02.xml
/home/eudoxos/oofem/tests/sm/DruckerPrager_01.in → ./xml/sm/DruckerPrager_01.xml
/home/eudoxos/oofem/tests/sm/EC2creep.in → ./xml/sm/EC2creep.xml
/home/eudoxos/oofem/tests/sm/EC2creep_casting.in → ./xml/sm/EC2creep_casting.xml
/home/eudoxos/oofem/tests/sm/EC2shrinkage.in → ./xml/sm/EC2shrinkage.xml
/home/eudoxos/oofem/tests/sm/InterfaceEL_Line1.in → ./xml/sm/InterfaceEL_Line1.xml
/home/eudoxos/oofem/tests/sm/Inter